In [12]:
CPU = '''
Time taken for K = 1: 0.00417109 seconds
Time taken for K = 5: 0.0191502 seconds
Time taken for K = 10: 0.0382319 seconds
Time taken for K = 50: 0.187489 seconds
Time taken for K = 100: 0.375146 seconds
'''

GPU_NO_UVM_1 = '''
Time taken for K = 1, Blocks = 1, Threads per Block = 1: 8.0846e-05 seconds
Time taken for K = 5, Blocks = 1, Threads per Block = 1: 8.4868e-05 seconds
Time taken for K = 10, Blocks = 1, Threads per Block = 1: 7.616e-05 seconds
Time taken for K = 50, Blocks = 1, Threads per Block = 1: 6.3845e-05 seconds
Time taken for K = 100, Blocks = 1, Threads per Block = 1: 5.1713e-05 seconds
'''

GPU_NO_UVM_2 = '''
Time taken for K = 1, Blocks = 1, Threads per Block = 256: 7.6181e-05 seconds
Time taken for K = 5, Blocks = 1, Threads per Block = 256: 9.3519e-05 seconds
Time taken for K = 10, Blocks = 1, Threads per Block = 256: 7.1892e-05 seconds
Time taken for K = 50, Blocks = 1, Threads per Block = 256: 6.5695e-05 seconds
Time taken for K = 100, Blocks = 1, Threads per Block = 256: 5.2135e-05 seconds
'''

GPU_NO_UVM_3 = '''
Time taken for K = 1, Blocks = 3907, Threads per Block = 256: 9.6069e-05 seconds
Time taken for K = 5, Blocks = 19532, Threads per Block = 256: 0.000190438 seconds
Time taken for K = 10, Blocks = 39063, Threads per Block = 256: 0.000291147 seconds
Time taken for K = 50, Blocks = 195313, Threads per Block = 256: 0.0011573 seconds
Time taken for K = 100, Blocks = 390625, Threads per Block = 256: 0.00225153 seconds
'''

GPU_UVM_1 = '''
Time taken for K = 1, Blocks = 1, Threads per Block = 1: 0.000364497 seconds
Time taken for K = 5, Blocks = 1, Threads per Block = 1: 0.000268716 seconds
Time taken for K = 10, Blocks = 1, Threads per Block = 1: 0.000280274 seconds
Time taken for K = 50, Blocks = 1, Threads per Block = 1: 0.000422188 seconds
Time taken for K = 100, Blocks = 1, Threads per Block = 1: 0.000337818 seconds
'''

GPU_UVM_2 = '''
Time taken for K = 1, Blocks = 1, Threads per Block = 256: 0.000456926 seconds
Time taken for K = 5, Blocks = 1, Threads per Block = 256: 0.000365698 seconds
Time taken for K = 10, Blocks = 1, Threads per Block = 256: 0.000470437 seconds
Time taken for K = 50, Blocks = 1, Threads per Block = 256: 0.000486186 seconds
Time taken for K = 100, Blocks = 1, Threads per Block = 256: 0.000506219 seconds
'''

GPU_UVM_3 = '''
Time taken for K = 1, Blocks = 3907, Threads per Block = 256: 0.00271336 seconds
Time taken for K = 5, Blocks = 19532, Threads per Block = 256: 0.00888042 seconds
Time taken for K = 10, Blocks = 39063, Threads per Block = 256: 0.0166929 seconds
Time taken for K = 50, Blocks = 195313, Threads per Block = 256: 0.0705386 seconds
Time taken for K = 100, Blocks = 390625, Threads per Block = 256: 0.127504 seconds
'''


In [21]:
import re
import pandas as pd

def parse_stats(stats):
    stats = stats.split('\n')
    time = {}
    for stat in stats:
        if not stat: continue
        k = int(re.findall(r'K = (\d+)', stat)[0])
        t = float(re.findall(r': (\d+\.\d+e?-?\d*)', stat)[0])
        time[k] = t
    return time

def make_stats_df(stats_dict):
    return pd.DataFrame(stats_dict).reset_index().rename(columns={'index': 'k'})


In [22]:
stats_dict = {}
for stat, name in zip([CPU, GPU_NO_UVM_1, GPU_NO_UVM_2, GPU_NO_UVM_3, GPU_UVM_1, GPU_UVM_2, GPU_UVM_3], ['CPU', 'GPU_NO_UVM_1', 'GPU_NO_UVM_2', 'GPU_NO_UVM_3', 'GPU_UVM_1', 'GPU_UVM_2', 'GPU_UVM_3']):
    stat = parse_stats(stat)
    stats_dict[name] = stat

df = make_stats_df(stats_dict)
df

,k,CPU,GPU_NO_UVM_1,GPU_NO_UVM_2,GPU_NO_UVM_3,GPU_UVM_1,GPU_UVM_2,GPU_UVM_3
0,1,0.004171,0.000081,0.000076,0.000096,0.000364,0.000457,0.002713
1,5,0.019150,0.000085,0.000094,0.000190,0.000269,0.000366,0.008880
2,10,0.038232,0.000076,0.000072,0.000291,0.000280,0.000470,0.016693
3,50,0.187489,0.000064,0.000066,0.001157,0.000422,0.000486,0.070539
4,100,0.375146,0.000052,0.000052,0.002252,0.000338,0.000506,0.127504


In [42]:
import numpy as np

def make_time_log_log(df):
    df['Time (sec)'] = np.log(df['Time (sec)']) / np.log(df['Time (sec)'].min())
    df['Time (sec)'] = np.log(df['Time (sec)']) / np.log(df['Time (sec)'].min())
    return df


In [47]:
df_no_uvm = pd.melt(df, id_vars=['k'], value_vars=['CPU', 'GPU_NO_UVM_1', 'GPU_NO_UVM_2', 'GPU_NO_UVM_3'], var_name='Machine/Scenario Type', value_name='Time (sec)')
df_no_uvm = make_time_log_log(df_no_uvm)

df_uvm = pd.melt(df, id_vars=['k'], value_vars=['CPU', 'GPU_UVM_1', 'GPU_UVM_2', 'GPU_UVM_3'], var_name='Machine/Scenario Type', value_name='Time (sec)')
df_uvm = make_time_log_log(df_uvm)


In [48]:
import plotly.express as px
import numpy as np

no_uvm_fig = px.scatter(df_no_uvm, x='k', y=df_no_uvm['Time (sec)'], color='Machine/Scenario Type', symbol='Machine/Scenario Type', title='NO UVM')
no_uvm_fig.update_yaxes(title='Time (sec) (log-log)')

uvm_fig = px.scatter(df_uvm, x='k', y=df_uvm['Time (sec)'], color='Machine/Scenario Type', symbol='Machine/Scenario Type', title='UVM')
uvm_fig.update_yaxes(title='Time (sec) (log-log)')


In [49]:
no_uvm_fig

In [50]:
uvm_fig